# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

#Q7
Implement a deep neural network, which contains two hidden layers (the hidden layers are obtained from
the ELM-autoencoders). The last layer will be the ELM layer which means the second hidden layer feature
vector is used as input to the ELM classifier. The network can be called as deep layer stacked autoencoder
based extreme learning machine. You can use holdout approach (70, 10, 20%) for evaluating the
performance of the classifier. The dataset (data5.xlsx) contains 7 features and the last column is the
output (class labels). Evaluate individual accuracy and overall accuracy. (Packages such as Scikitlearn,
keras, tensorflow, pytorch etc. are not allowed)

In [ ]:
def sigmoidFunction(Z):
    return 1/(1+np.exp(-Z)),Z

def relu(Z):
    A = np.maximum(0,Z)
    return Z,A

def reluBackward(dA, cache):
    Z = cache
    
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoidBackward(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    return dZ

def initializeNewLayer(layer_dims,para,stack):
    parameters = {}
    L = len(layer_dims)            
    for l in range(1, L-1):
        if stack==False:
            parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])
        else:
            parameters['W' + str(l)]=para[l-1]['W1']
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    parameters['W' + str(L-1)] = np.random.randn(layer_dims[L-1],layer_dims[L-2])
    parameters['b' + str(L-1)] = np.zeros((layer_dims[L-1],1))
    return parameters

def linearForward(A, W, b):
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    return Z,cache

def linearActivationFunction(A_prev, W, b, activation):
    
    if(activation == "sigmoid"):
        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = sigmoidFunction(Z)
    
    elif(activation == "relu"):
        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

def modelLForward(X, params):
    caches = []
    A = X
    L = (len(params) //2)
    for l in range(1, L):
        A_prev = A 
        A, cache = linearActivationFunction(A, params['W'+str(l)], params['b'+str(l)],"sigmoid")
        caches.append(cache)
    
    AL, cache = linearActivationFunction(A, params['W'+str(L)], params['b'+str(L)],"sigmoid")
    caches.append(cache)

    return AL,caches

def costComputation(AL,Y):
    m = Y.shape[1]
    cost = -(1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

def costComputationAE(AL,Y,parameters):
    m=Y.shape[1]
    cost=(1/(2*m))*np.sum(np.linalg.norm(AL-Y))
    L = len(parameters) //2
    return cost

def linearBack(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
   
    return dA_prev, dW, db

def linearActivation(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = reluBackward(dA,activation_cache)
        dA_prev, dW, db = linearBack(dZ,linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoidBackward(dA,activation_cache)
        dA_prev, dW, db = linearBack(dZ,linear_cache)
    
    return dA_prev, dW, db

def backModelL(AL, Y, caches):
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linearActivation(dAL,current_cache,"sigmoid")
   
    for l in reversed(range(L-1)):
    
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linearActivation(grads["dA"+str(l+1)],current_cache,"sigmoid")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def updateParams(params, grads, learning_rate):
   
    L = len(params) //2

    for l in range(L):
        params["W" + str(l+1)] = params["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        params["b" + str(l+1)] = params["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
  
    return params

def layerLModel(X, Y, layers_dims, num_iterations,stack, learning_rate = 0.025):

    costs = []                        
 
    parameters = initializeNewLayer(layers_dims,para,stack)

    for i in range(0, num_iterations):

        AL, caches = modelLForward(X,parameters)
        
        if stack==True:
            cost = costComputation(AL,Y)
        else:
            cost = costComputationAE(AL,Y,parameters)
        
        grads = backModelL(AL,Y,caches)
     
        parameters = updateParams(parameters,grads,learning_rate)            
    
    return (parameters, costs)

para = []
def ACT(x, a, b, act):
    if(act == "gaussian"):
        return np.exp(-b*np.linalg.norm(x-a))
    elif(act == "tanh"):
        num = 1 - np.exp(-(np.dot(x.T, a) + b))
        den = 1 + np.exp(-(np.dot(x.T, a) + b))
        return (num/den)

def init(l_hidden, dimensions):
    a = []
    b = []
    for i in range(l_hidden):
        a.append(np.random.rand(dimensions,1))
        b.append(np.random.rand(1))
    return (a,b)

def oneHotEnc(y):
    from sklearn.preprocessing import OneHotEncoder 
    onehotencoder = OneHotEncoder() 
    y = onehotencoder.fit_transform(y).toarray()
    return y

def compute(l_hidden,training_data_X,testing_data_X,training_data_y,testing_data_y,act):

    Y_enc = oneHotEnc(training_data_y)
    H = np.zeros((training_data_X.shape[0],l_hidden))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=ACT(train_x[j],a[i],b[i],act)
    W=np.dot(np.linalg.pinv(H),Y_enc)
    
    H=np.zeros((testing_data_X.shape[0],l_hidden))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=ACT(testing_data_X[j],a[i],b[i],act)

    p = np.dot(H,W)
    p = np.argmax(p,axis=1)
    p = np.reshape(p,newshape=(p.shape[0],1))
    metrics(p, testing_data_y)


dataset = pd.read_excel('data5.xslx', header = None)

row, col = dataset.shape
feats = col - 1 

# normalization
dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)

# spliting dataset into train test and val
training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

training_data = np.array(training_data)
validation_data = np.array(validation_data)
testing_data = np.array(testing_data)
training_data_X = training_data[:, :feats]
training_data_y = training_data[:, feats]
validation_data_X = validation_data[:, :feats]
validation_data_y = validation_data[:, feats]
testing_data_X = testing_data[:, :feats]
testing_data_y = testing_data[:, feats]

train_row, train_col = training_data_X.shape

layers_dims = [72,32,72]
parameters, costs = layerLModel(training_data_X, training_data_y, layers_dims, num_iterations=1500, stack=False, learning_rate=0.005)
paramsAE1 = parameters

W1 = paramsAE1['W1']
b1 = paramsAE1['b1']
X_new, _ = linearActivationFunction(X.T,W1,b1,"sigmoid")

layers_dims = [32,16,32]
parameters,costs = layerLModel(training_data_X, training_data_y, layers_dims,num_iterations=1500,stack=False,learning_rate=0.05, print_cost = True)
paramsAE2 = parameters

W1 = paramsAE1['W1']
W2 = paramsAE2['W1']
b1 = paramsAE1['b1']
b2 = paramsAE2['b1']

x, _ = sigmoidFunction(np.dot(W1,X.T) + b1)
x, _ = sigmoidFunction(np.dot(W2,x) + b2)

a, b = init(256, X.shape[1])
print("Tanh Accuracy: ")
compute(256, training_data_X, testing_data_X, training_data_y, testing_data_y, "tanh")
print()
a, b = init(256,X.shape[1])
print("Gaussian Accuracy: ")
compute(256,training_data_X,testing_data_X,training_data_y,testing_data_y,"gaussian")


Tanh Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.8181818181818182
Specificity :  0.8888888888888888
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.855072463768116

Gaussian Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.9230769230769231
Specificity :  0.851063829787234
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8837209302325582
